In [ ]:
from data_generators import get_train_test_addition

#dimensjoner og størrelser til x og y
n_digits = 2
n_max = 3*n_digits
m = 10

#definerer størrelsen på parametermatrisene
d = 15
k = 5
p = 20

#henter treningsdata
data = get_train_test_addition(n_digits,samples_per_batch=100,n_batches_train=4)

In [ ]:
from utils import onehot

x = data['x_train'][0]
X = onehot(x,m)
y = data['y_train'][0]

In [ ]:
data['x_train'].shape

In [ ]:
from layers import LinearLayer,EmbedPosition,Attention,Softmax,CrossEntropy,FeedForward

embed = EmbedPosition(n_max,m,d)
att1 = Attention(d,k)
ff1 = FeedForward(d,p)
un_embed = LinearLayer(d,m)
softmax = Softmax()
loss = CrossEntropy()

In [ ]:
#vi ser at dLdW_2 tilhørende ff1 (først finner vi det lineære laget l2
#så nøkkel 'w' i params dict-en, så nøkkel 'd' for å finne den deriverte)
#nå er denne bare en nullmatrise 
ff1.l2.params['w']['d']

In [ ]:
#"manuelt" forward pass (tilsvarende algoritme 1)
z0 = embed.forward(X)
z1 = att1.forward(z0)
z2 = ff1.forward(z1)
z = un_embed.forward(z2)
Z = softmax.forward(z)

In [ ]:
#evaluerer objektfunksjonen
L = loss.forward(Z,y)
print(L)

#finner den deriverte av objektfunksjonen mhp Z
dLdZ = loss.backward()

In [ ]:
#"manuelt" backward pass (tilsvarende algoritme 2)
dLdz = softmax.backward(dLdZ)
dLdz2 = un_embed.backward(dLdz)
dLdz1 = ff1.backward(dLdz2)
dLdz0 = att1.backward(dLdz1)
embed.backward(dLdz0)

In [ ]:
#etter backward pass har dLdW_2 fått verdier
ff1.l2.params['w']['d']

In [ ]:
from neural_network import NeuralNetwork

#vi kan samle lagene i en liste som vi bruker for å
#initialisere et nevralt nettverk der vi kan bruke forward() og backward() 
#for å oppnå det samme som vi gjorde manuelt over

layers = [embed,att1,ff1,un_embed,softmax]
nn = NeuralNetwork(layers)

In [ ]:
#forward pass tilsvarende algoritme 1
Z = nn.forward(X)

#beregner loss med CrossEntropy
L = loss.forward(Z,y)
print(L)

#backward pass tilsvarende algoritme 2
dLdZ = loss.backward()
nn.backward(dLdZ)

In [ ]:
#før optimering er W_2[0,0] gitt ved
#ff1 er det tredje laget i layers-listen (derav layers[2])
W_2_pre_opt = nn.layers[2].l2.params['w']['w'].copy()
print(W_2_pre_opt[0,0])

In [ ]:
import numpy as np

xs = data['x_train']
ys = data['y_train']

n_batches = xs.shape[0]
n_iters = 100
step_size = 0.1

#treningsløkke tilsvarende algoritme 4 (med gradient descent)
for j in range(n_iters):
    losses = []
    for i in range(n_batches):
        x = xs[i]
        y = ys[i]

        X = onehot(x,m)
        Z = nn.forward(X)

        losses.append(loss.forward(Z,y))
        dLdZ = loss.backward()
        nn.backward(dLdZ)
        nn.step_gd(step_size)
    mean_loss = np.mean(losses)
    print("Iterasjon ", str(j), " L = ",mean_loss, "")

In [ ]:
import numpy as np

xs = data['x_train']
ys = data['y_train']

n_batches = xs.shape[0]
n_iters = 100
step_size = 0.01

#treningsløkke tilsvarende algoritme 4 (med gradient descent)
for j in range(n_iters):
    losses = []
    for i in range(n_batches):
        x = xs[i]
        y = ys[i]

        X = onehot(x,m)
        Z = nn.forward(X)

        losses.append(loss.forward(Z,y))
        dLdZ = loss.backward()
        nn.backward(dLdZ)
        nn.step_adam(step_size, j + 1)
    mean_loss = np.mean(losses)
    print("Iterasjon ", str(j), " L = ",mean_loss, "")

In [ ]:
#etter trening ser vi at W_2 er gitt ved

W_2_post_opt = nn.layers[2].l2.params['w']['w'].copy()
print(W_2_post_opt[0,0])

#dersom differansen er større enn null har dette parameteret endret seg etter 
#gradient descent
print(W_2_post_opt[0,0] - W_2_pre_opt[0,0])


